In [ ]:
!wget https://raw.githubusercontent.com/ygtxr1997/CelebBasis/main/infer_images/wiki_names.txt

In [ ]:
!wget https://raw.githubusercontent.com/ygtxr1997/CelebBasis/main/infer_images/celebs.txt

In [ ]:
from pathlib import Path

#with Path('wiki_names.txt').open() as f:
with Path('celebs.txt').open() as f:
    wiki_names = f.read().split('\n')

len(wiki_names) # 1109

first_names = [n for n in set([name.split(' ')[0] for name in wiki_names]) if '.' not in n]
last_names = [n for n in set([name.split(' ')[-1] for name in wiki_names]) if '.' not in n]

len(first_names), len(last_names)

In [ ]:
first_names[:10]

In [ ]:
last_names[:10]

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "CompVis/stable-diffusion-v1-4"

pipe = StableDiffusionPipeline.from_pretrained(
                model_id,
                revision="fp16",
                #torch_dtype=torch.float16,
                #use_auth_token=True
            ).to('cuda')

In [ ]:
#"diffusers",
#    "EulerDiscreteScheduler"
#pipe.scheduler
import diffusers
#pipe.scheduler = diffusers.EulerDiscreteScheduler()
# https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/scheduler/scheduler_config.json
true, false, null = True, False, None
pipe.scheduler = diffusers.EulerDiscreteScheduler(
    **{
      #"_class_name": "EulerDiscreteScheduler",
      #"_diffusers_version": "0.19.0.dev0",
      "beta_end": 0.012,
      "beta_schedule": "scaled_linear",
      "beta_start": 0.00085,
      #"clip_sample": false,
      "interpolation_type": "linear",
      "num_train_timesteps": 1000,
      "prediction_type": "epsilon",
      #"sample_max_value": 1.0,
      #"set_alpha_to_one": false,
      #"skip_prk_steps": true,
      "steps_offset": 1,
      "timestep_spacing": "leading",
      "trained_betas": null,
      "use_karras_sigmas": false
    }
)

In [ ]:
diffusers.EulerDiscreteScheduler()

In [ ]:
kwargs = dict(
    num_inference_steps=25,
    num_images_per_prompt=1,
    
)
prompt = "a painting of an apple"
outv = pipe(prompt, **kwargs)

In [ ]:
outv.images[0]

In [ ]:
# 1. loop over names
# 2. create folder for names
# 3. generate images, save to folder
# 4. compute CLIP/DINO image embeddings, save to folder

# ---

# let's start with 10 names and prototype this procedure generally.
# an important hyper parameter to start with will be the number of images per name.
# let's start by generating an unnecessary number of images for this first group, say 20-100.
# then use these to compute our diversity measures, and calibrate how many images we need by how many images it generally takes to get this measure to converge.


In [ ]:
#pipe.requires_safety_checker=False

In [ ]:
def passthrough(images, *args, **kargs):
    return images, [False for _ in images]
pipe.safety_checker = passthrough

In [ ]:
import time

#names = ["david"] ## TODO: dev
names = first_names

kwargs = dict(
    num_inference_steps=25,
    num_images_per_prompt=8
)

root = Path("images")

images_per_prompt = 24 #40

for name in names:
    #prompt = name
    print(name)
    prompt = f"a photo of {name}, portrait photography, full color, face full frame"
    prompt = prompt.lower().strip()
    
    prompt_clean = prompt.replace(',','')
    outdir = root / prompt_clean.replace(' ','_')  
    outdir.mkdir(parents=True, exist_ok=True)
    while len(list(outdir.glob('*.png'))) < images_per_prompt:
        outv = pipe(prompt, **kwargs)
        if any(outv.nsfw_content_detected):
            #continue
            break

        for idx,im in enumerate(outv.images):
            # would prefer to name images by seed, but hf is makes it hard.
            fname = outdir / f"{int(time.time())}_{idx}.png" 
            im.save(fname)


In [ ]:
# let's generate a shit load of images 
# for a few high-bias names as a calibration control
# for sensitivity analysis.

high_bias_names = [
    "oprah",
    "keanu",
    "cher",
    "beyoncé",
    "shaquille",
    "rihanna",
    "anupam",
]

images_per_prompt = 200


for name in high_bias_names:
    #prompt = name
    print(name)
    prompt = f"a photo of {name}, portrait photography, full color, face full frame"
    prompt = prompt.lower().strip()
    
    prompt_clean = prompt.replace(',','')
    outdir = root / prompt_clean.replace(' ','_')  
    outdir.mkdir(parents=True, exist_ok=True)
    while len(list(outdir.glob('*.png'))) < images_per_prompt:
        outv = pipe(prompt, **kwargs)
        if any(outv.nsfw_content_detected):
            #continue
            break

        for idx,im in enumerate(outv.images):
            # would prefer to name images by seed, but hf is makes it hard.
            fname = outdir / f"{int(time.time())}_{idx}.png" 
            im.save(fname)

# TODO

* filtter out images with more than one face in them (check and report on what gets pulled out)
* calibrate # images for experiments
  * "power analysis" - how many images do we need to detect the variance threshold we're interested in?"
* DINOv2 embeddings
* per-prompt (cluster) summary statistics
* tSNE
* separately generate and save input clip emberddings of prompts
* score images relative to centroid in addition to summary statistic for cluster
* repeat procedure across several checkpoints, compare per-cluster summary statistics across checkpoints. expect to observe narrowing of biases as checkpoints are progressively finetuned.

In [ ]:
# DINOv2 embeddings